# Introduction

Author: Johannes Peter Knoll

Within this notebook you will learn and test everything that was implemented to preprocess the data
for the neural network.

Note:   This notebook is rather for those who want to make sure everything works correctly. It is very thorough
        and therefore unnecessary if you only want to get a quick start into the predictions. If that is the case, head
        to 'Classification_Demo.ipynb'


# Thorough Demonstration of 'dataset_processing.py'

In [1]:
# The autoreload extension allows you to tweak the code in the imported modules
# and rerun cells to reflect the changes.
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Managing Data

In this section we demonstrate the implemented class that helps you to manage the data you want to pass to the
neural network model.

The usage of this class is not required, you could also just use the implemented functions on your data, which
are explained in the next section.

I still would highly recommend using this class, as it is able to handle large data in a memory saving way
and makes it very easy to check and process your data, so that it can be passed to the model easily.

In [2]:
from dataset_processing import *

import random
import copy

### Basics

The Database itself is just a .pkl file that contains multiple dictionaries. The first dictionary is always
the file information, while the following will be the Database's datapoints. Each datapoint needs a unique ID
(key: "ID") and can contain the following data: RRI-Signal (key: "RRI"), MAD-Signal  (key: "MAD"), Sleep-Labels 
(key: "SLP") and their transformation to overlapped windows: (keys: "RRI_windows", "MAD_windows", "SLP_windows").

You might wonder about the individual sampling frequencies... \
The point of the database is to store the data in the same format, so that it can be processed by the neural network
correctly. Therefore the sampling frequency for each of the signals is uniform across all datapoints.
(For our network we need: RRI frequency = 4, Mad frequency = 1, and SLP frequency = 1/30 for all datapoints.)

The frequency for each signal is saved in the file information. 

### Note

I do not know if this needs to be said, but NEVER MANUALLY CHANGE THE CLASS ATTRIBUTES when data was already
added.

If you have different requirements for the unfiform frequency, signal length or any of the other parameters,
that's fine. But change them before you save data to it. Even better: Alter the hard coded default values in
the class, so that you do not forget to update the file information each time you create a new instance of 
the class.

### Creating Database (.pkl - file)

When initializing a database (calling 'SleepDataManager' on non-existent path to .pkl file) the class will
automatically write the first dictionary to it, which functions as an information on the data properties
within the file.

In [3]:
some_data_manager = SleepDataManager(file_path = "Sleep_Data/messing_around.pkl")
file_information = some_data_manager.file_info

for key in file_information.keys():
    print(f"{key}: {file_information[key]}")

RRI_frequency: 4
MAD_frequency: 1
SLP_frequency: 0.03333333333333333
sleep_stage_label: {'wake': 0, 'LS': 1, 'DS': 2, 'REM': 3, 'artifect': -1}
signal_length_seconds: 36000
wanted_shift_length_seconds: 3600
absolute_shift_deviation_seconds: 1800
signal_in_windows: False
number_windows: 1197
window_duration_seconds: 120
overlap_seconds: 90
pad_target_with: 0
pad_feature_with: 0
priority_order: [0, 1, 2, 3, 5, -1]
train_val_test_split_applied: False
main_file_path: Sleep_Data/messing_around.pkl
train_file_path: Sleep_Data/messing_around_training_pid.pkl
validation_file_path: Sleep_Data/messing_around_validation_pid.pkl
test_file_path: Sleep_Data/messing_around_test_pid.pkl


We already mentioned the first three keys, don't mind the others yet. Most of them will be explained below.
For the few others, take a look at the functions: 'reshape_signals_to_overlapping_windows' and 'split_long_signal'.

### Saving Data

Saving Data is harder than it sounds. As mentioned above: we want to store uniform data. Therefore we need to 
alter the data so that it fits to the file data parameters (file info).

By "we" I of course mean the simple function that you call to save data. I just wanted to point out that 
it does not trust you on the data you pass to it. So, if you want to add a signal, you must also pass its
sampling frequency (keys: "RRI_frequency", "MAD_frequency", "SLP_frequency").

The only keys you are allowed to provide are the following: "ID", "RRI", "MAD", "SLP", "RRI_frequency", 
"MAD_frequency", "SLP_frequency" and "sleep_stage_label". Yes, above we said the datapoint can also contain "RRI_windows", 
"MAD_windows" or "SLP_windows". But these will be created from the class, you are not allowed to pass this 
data manually.

The save() function takes the new data as dictionary and also has an optional argument: 'overwrite_id' (bool).
If set to 'True' it will look for the same "ID" in the database and update the signals. If set to 'False' it
will not append the data, if the same ID already exists in the database. This is advantageous, because if
you do not create the "ID" yourself, then you won't accidentally overwrite one datapoint with a different one.

Operations that might be happening to the data you try to save:
- scale number of datapoints in signal so that signal frequency matches uniform database signal frequency
- alter sleep labels
- split signal into multiple signals if signal is longer than the uniform maximum signal length: 'signal_length_seconds'

Altering the sleep stage labels is highly important and must therefore be dealt with caution, that's why I
will provide an example here:

In [4]:
# sleep stage labels in shhs dataset:
shhs_labels = {"wake": 0, "N1": 1, "N2": 2, "N3": 3, "REM": 5, "artifect": "other"}

# in the nn we only divide between wake, LS, DS, REM, and artifect. Above, N1 must be redeclared as "wake", 
# N2 as "LS" and N3 as "DS":
shhs_labels = {"wake": [0, 1], "LS": [2], "DS": [3], "REM": [5], "artifect": ["other"]}

The values for the keys must be lists, this was implemented for the special case above, in which the sleep
stages were more subdivided than in the neural network.

Now we will pass the worst possible data to the database: differing sampling frequencies and overlength:

In [5]:
# creating signal with different sampling frequencies and overlength:
signal_time_in_seconds = 12.1 * 3600
rri_frequency = 6
mad_frequency = 2
slp_frequency = 1/20

# creating signals and printing manually scaled versions
rri_signal = np.array([random.randint(1, 5) for i in range(int(signal_time_in_seconds * rri_frequency))], dtype=np.float64)
print(f"First datapoints of RRI signal: {rri_signal[:10]} (shape: {rri_signal.shape})")
interpolate_rri = interpolate_signal(rri_signal, rri_frequency, 4) # type: ignore
print(f"First datapoints of RRI signal scaled: {interpolate_rri[:10]} (shape: {interpolate_rri.shape})")
mad_signal = [random.randint(1, 5) for i in range(int(signal_time_in_seconds * mad_frequency))]
print(f"First datapoints of MAD signal: {mad_signal[:10]} (shape: {len(mad_signal)})")
interpolate_mad = interpolate_signal(mad_signal, mad_frequency, 1) # type: ignore
print(f"First datapoints of MAD signal scaled: {interpolate_mad[:10]} (shape: {len(interpolate_mad)})")
slp_signal = [random.randint(1, 5) for i in range(int(signal_time_in_seconds * slp_frequency))]
print(f"First datapoints of SLP signal: {slp_signal[:10]} (shape: {len(slp_signal)})")
scaled_slp = scale_classification_signal(slp_signal, slp_frequency, 1/30) # type: ignore
print(f"First datapoints of SLP signal scaled: {scaled_slp[:10]} (shape: {len(scaled_slp)})")

random_sleep_stage_labels = {"wake": [0, 1], "LS": [2], "DS": [3], "REM": [5], "artifect": ["other"]}
altered_scaled_slp = alter_slp_labels(scaled_slp, random_sleep_stage_labels) # type: ignore
print(f"First datapoints of scaled SLP signal altered: {altered_scaled_slp[:10]} (shape: {len(altered_scaled_slp)})")

new_datapoint = {
    "ID": "4",
    "RRI": rri_signal,
    "RRI_frequency": rri_frequency,
    "MAD": mad_signal,
    "MAD_frequency": mad_frequency,
    "SLP": slp_signal,
    "SLP_frequency": slp_frequency,
    "sleep_stage_label": random_sleep_stage_labels
}

First datapoints of RRI signal: [4. 4. 1. 4. 4. 1. 2. 4. 1. 5.] (shape: (261360,))
First datapoints of RRI signal scaled: [4.  2.5 4.  2.5 2.  2.5 5.  1.5 5.  2.5] (shape: (174240,))
First datapoints of MAD signal: [3, 3, 5, 4, 2, 1, 3, 5, 3, 1] (shape: 87120)
First datapoints of MAD signal scaled: [3 5 2 3 3 3 2 5 2 1] (shape: 43560)
First datapoints of SLP signal: [3, 4, 5, 5, 3, 3, 3, 5, 5, 1] (shape: 2178)
First datapoints of SLP signal scaled: [3 4 5 3 3 5 1 3 2 4] (shape: 1452)
First datapoints of scaled SLP signal altered: [ 2 -1  3  2  2  3  0  2  1 -1] (shape: 1452)


In [6]:
# saving the new datapoints
some_data_manager.save(copy.deepcopy(new_datapoint), overwrite_id=True, unique_id=False)

In [7]:
# print the data
for dict in some_data_manager:
    print("-"*70)
    for key in dict.keys():
        if key in ["RRI", "MAD", "SLP"]:
            print(key, dict[key][:10], dict[key].shape)
        else:
            print(key, dict[key])
print("-"*70)

----------------------------------------------------------------------
ID 4
RRI [4.  2.5 4.  2.5 2.  2.5 5.  1.5 5.  2.5] (144000,)
MAD [3 5 2 3 3 3 2 5 2 1] (36000,)
SLP [ 2 -1  3  2  2  3  0  2  1 -1] (1200,)
----------------------------------------------------------------------
ID 4_shift_3780s_x1
RRI [5.  2.  3.  3.5 2.  1.5 2.  4.5 2.  2.5] (144000,)
MAD [4 4 3 4 1 3 2 1 1 1] (36000,)
SLP [2 3 2 0 1 3 0 0 2 0] (1200,)
----------------------------------------------------------------------
ID 4_shift_3780s_x2
RRI [1.  3.  1.  3.  4.  3.  5.  1.5 1.  4. ] (144000,)
MAD [5 5 2 1 3 3 3 4 5 4] (36000,)
SLP [-1  3 -1  3  3  0  2  2  3  2] (1200,)
----------------------------------------------------------------------


Don't worry at this point too much about how the scaling and splitting works. The responsible functions are
explained in more detail (far) below. 

Just notice that the signal length now matches the uniform file length of 10 hours (36 000 seconds).
(Remember: number datapoints in signal = signal length in seconds * frequency).

You might also see that the signals in the first splitted signal ("ID"=4) match the manually rescaled
signals in the code above where we initialized the new datapoint.

If you want you can now try how the 'overwrite_id' behaves. Run the 3 cells above again, with different values
for it and inspect the changes.

If 'True', you will see that the signals will change.

If 'False', the program will raise an error, telling you to check the "ID" (Don't worry, your data will not be
lost afterwards). The signals will remain the same.

#### Speed up data saving

Because every id needs to be checked, it takes a while to compute if you want to add many datapoints.

To speed up the process, you can check once if every id is unique and afterwards skip the checking when
adding the datapoints to the database.

In [8]:
list_of_ids = ["101", "102", "103"]

some_data_manager.check_if_ids_are_unique(list_of_ids)

All IDs are unique.


If all ids are unique you can use: some_data_manager(..., unique_id=True) and the saving will be much faster!

### Load Data

Data can be loaded in multiple ways. First we will look at the load() function.
The argument you pass to it ('key_id_index') can be a string or an integer. 

If it's an integer, it will treat it as position in the database, analogue to a list. It will return the
whole data dictionary. \
If it's a string that equals a data key it will return all entities of that specific key in the database. \
If it's a different string, then it will treat it as ID and look for a match. Equal to index, it will return
the whole dictionary.

In [9]:
loaded_data = some_data_manager.load(1)
print(loaded_data)

{'ID': '4_shift_3780s_x1', 'RRI': array([5. , 2. , 3. , ..., 4.5, 5. , 1. ]), 'MAD': array([4, 4, 3, ..., 4, 5, 2]), 'SLP': array([2, 3, 2, ..., 2, 0, 0])}


In [10]:
loaded_data = some_data_manager.load("4_shift_3780s_x1")
print(loaded_data)

{'ID': '4_shift_3780s_x1', 'RRI': array([5. , 2. , 3. , ..., 4.5, 5. , 1. ]), 'MAD': array([4, 4, 3, ..., 4, 5, 2]), 'SLP': array([2, 3, 2, ..., 2, 0, 0])}


In [11]:
loaded_data = some_data_manager.load("RRI")
print(loaded_data)

[array([4. , 2.5, 4. , ..., 3.5, 4. , 1.5]), array([5. , 2. , 3. , ..., 4.5, 5. , 1. ]), array([1., 3., 1., ..., 2., 4., 3.])]


A second way of achieving the last result is by calling:

In [12]:
loaded_data = some_data_manager["MAD"]
print(loaded_data)

[array([3, 5, 2, ..., 1, 1, 3]), array([4, 4, 3, ..., 4, 5, 2]), array([5, 5, 2, ..., 3, 4, 4])]


### Remove Data

remove() takes the same key as load() and works analogue, but instead of loading it will remove from the database.

In [13]:
some_data_manager.remove("RRI")

# print all data
for dict in some_data_manager:
    print("-"*20)
    for key in dict.keys():
        if key in ["RRI", "MAD", "SLP"]:
            print(key, dict[key].shape)
        else:
            print(key, dict[key])
print("-"*20)

--------------------
ID 4
MAD (36000,)
SLP (1200,)
--------------------
ID 4_shift_3780s_x1
MAD (36000,)
SLP (1200,)
--------------------
ID 4_shift_3780s_x2
MAD (36000,)
SLP (1200,)
--------------------


In [14]:
some_data_manager.remove("4_shift_3780s_x1")

# print all data
for dict in some_data_manager:
    print("-"*20)
    for key in dict.keys():
        if key in ["RRI", "MAD", "SLP"]:
            print(key, dict[key].shape)
        else:
            print(key, dict[key])
print("-"*20)

--------------------
ID 4
MAD (36000,)
SLP (1200,)
--------------------
ID 4_shift_3780s_x2
MAD (36000,)
SLP (1200,)
--------------------


In [15]:
some_data_manager.remove(0)

# print all data
for dict in some_data_manager:
    print("-"*20)
    for key in dict.keys():
        if key in ["RRI", "MAD", "SLP"]:
            print(key, dict[key].shape)
        else:
            print(key, dict[key])
print("-"*20)

--------------------
ID 4_shift_3780s_x2
MAD (36000,)
SLP (1200,)
--------------------


For now, let's restore the data:

In [16]:
some_data_manager.save(copy.deepcopy(new_datapoint), overwrite_id=True, unique_id=False)

### Other 'smaller' operations

Iterating over Database:

In [17]:
for datapoint in some_data_manager:
    print(datapoint["ID"])    

4_shift_3780s_x2
4
4_shift_3780s_x1


Checking if datapoint with certain ID is in database:

In [18]:
if "4" in some_data_manager:
    print("Datapoint with \"ID\" = 4 is in the data manager")

Datapoint with "ID" = 4 is in the data manager


print function:

In [19]:
print(some_data_manager)

file_path: Sleep_Data/messing_around.pkl
file_info: {'RRI_frequency': 4, 'MAD_frequency': 1, 'SLP_frequency': 0.03333333333333333, 'sleep_stage_label': {'wake': '0', 'LS': '1', 'DS': '2', 'REM': '3', 'artifect': '-1'}, 'signal_length_seconds': 36000, 'wanted_shift_length_seconds': 3600, 'absolute_shift_deviation_seconds': 1800, 'signal_in_windows': False, 'number_windows': 1197, 'window_duration_seconds': 120, 'overlap_seconds': 90, 'pad_target_with': 0, 'pad_feature_with': 0, 'priority_order': [0, 1, 2, 3, 5, -1], 'train_val_test_split_applied': False, 'main_file_path': 'Sleep_Data/messing_around.pkl', 'train_file_path': 'Sleep_Data/messing_around_training_pid.pkl', 'validation_file_path': 'Sleep_Data/messing_around_validation_pid.pkl', 'test_file_path': 'Sleep_Data/messing_around_test_pid.pkl'}


### Transforming signals into windows

We do not feed our neural network model with the whole 10 hour data. Instead, we transform our signals into
windows of a certain size that overlap. This way, we get multiple predictions for one time slot. More on
the topic and how it works (far) below.

For now, we only want to know how we do it for the database:

In [20]:
file_info_before_transforming = copy.deepcopy(some_data_manager.file_info)

some_data_manager.transform_signals_to_windows(
    number_windows = 1197, 
    window_duration_seconds = 120, 
    overlap_seconds = 90, 
    priority_order = [0, 1, 2, 3, 5, -1]
    )

file_info_after_transforming = copy.deepcopy(some_data_manager.file_info)

Let's check the output:

In [21]:
# print all data
for dict in some_data_manager:
    print("-"*70)
    for key in dict.keys():
        if key in ["RRI", "MAD", "SLP"]:
            print(key, dict[key][:10], dict[key].shape)
        elif key in ["RRI_windows", "MAD_windows"]:
            print(key, dict[key][0][:10], dict[key].shape)
        elif key in ["SLP_windows"]:
            print(key, dict[key][0], dict[key].shape)
        else:
            print(key, dict[key])
print("-"*70)

----------------------------------------------------------------------
ID 4_shift_3780s_x2
MAD [5 5 2 1 3 3 3 4 5 4] (36000,)
SLP [-1  3 -1  3  3  0  2  2  3  2] (1200,)
RRI [1.  3.  1.  3.  4.  3.  5.  1.5 1.  4. ] (144000,)
RRI_windows [1.  3.  1.  3.  4.  3.  5.  1.5 1.  4. ] (1197, 480)
MAD_windows [5. 5. 2. 1. 3. 3. 3. 4. 5. 4.] (1197, 120)
SLP_windows 3.0 (1197,)
----------------------------------------------------------------------
ID 4
RRI [4.  2.5 4.  2.5 2.  2.5 5.  1.5 5.  2.5] (144000,)
MAD [3 5 2 3 3 3 2 5 2 1] (36000,)
SLP [ 2 -1  3  2  2  3  0  2  1 -1] (1200,)
RRI_windows [4.  2.5 4.  2.5 2.  2.5 5.  1.5 5.  2.5] (1197, 480)
MAD_windows [3. 5. 2. 3. 3. 3. 2. 5. 2. 1.] (1197, 120)
SLP_windows 2.0 (1197,)
----------------------------------------------------------------------
ID 4_shift_3780s_x1
RRI [5.  2.  3.  3.5 2.  1.5 2.  4.5 2.  2.5] (144000,)
MAD [4 4 3 4 1 3 2 1 1 1] (36000,)
SLP [2 3 2 0 1 3 0 0 2 0] (1200,)
RRI_windows [5.  2.  3.  3.5 2.  1.5 2.  4.5 2.  2.5] (

Besides the change in the datapoints, this transformation also changed the file information:

In [22]:
for key in file_info_before_transforming.keys():
    print(f"{key} (old / new): {file_info_before_transforming[key]} / {file_info_after_transforming[key]}")

RRI_frequency (old / new): 4 / 4
MAD_frequency (old / new): 1 / 1
SLP_frequency (old / new): 0.03333333333333333 / 0.03333333333333333
sleep_stage_label (old / new): {'wake': '0', 'LS': '1', 'DS': '2', 'REM': '3', 'artifect': '-1'} / {'wake': '0', 'LS': '1', 'DS': '2', 'REM': '3', 'artifect': '-1'}
signal_length_seconds (old / new): 36000 / 36000
wanted_shift_length_seconds (old / new): 3600 / 3600
absolute_shift_deviation_seconds (old / new): 1800 / 1800
signal_in_windows (old / new): False / True
number_windows (old / new): 1197 / 1197
window_duration_seconds (old / new): 120 / 120
overlap_seconds (old / new): 90 / 90
pad_target_with (old / new): 0 / 0
pad_feature_with (old / new): 0 / 0
priority_order (old / new): [0, 1, 2, 3, 5, -1] / [0, 1, 2, 3, 5, -1]
train_val_test_split_applied (old / new): False / False
main_file_path (old / new): Sleep_Data/messing_around.pkl / Sleep_Data/messing_around.pkl
train_file_path (old / new): Sleep_Data/messing_around_training_pid.pkl / Sleep_Data/

Even if you see only one change (in 'signal_in_windows'), the parameters passed to the function will also be changed if they differ
from the file information: 'number_windows', 'window_duration_seconds', 'overlap_seconds', 'priority_order'. This is, of course,
because we want to keep our data uniform.

As mentioned: 'signal_in_windows' changed to 'True'. From now on, if you save more data to your
database, it will automatically add the transformed version to each signal:

In [23]:
# optimal signal (fitting sampling frequencies and length):
signal_time_in_seconds = 10 * 3600
rri_frequency = 4
mad_frequency = 1
slp_frequency = 1/30

rri_signal = np.array([random.randint(1, 5) for i in range(int(signal_time_in_seconds * rri_frequency))], dtype=np.float64)
mad_signal = [random.randint(1, 5) for i in range(int(signal_time_in_seconds * mad_frequency))]
slp_signal = [random.randint(1, 5) for i in range(int(signal_time_in_seconds * slp_frequency))]

random_sleep_stage_labels = {"wake": [0, 1], "LS": [2], "DS": [3], "REM": [5], "artifect": ["other"]}

new_datapoint = {
    "ID": "miwow",
    "RRI": rri_signal,
    "RRI_frequency": rri_frequency,
    "MAD": mad_signal,
    "MAD_frequency": mad_frequency,
    "SLP": slp_signal,
    "SLP_frequency": slp_frequency,
    "sleep_stage_label": random_sleep_stage_labels
}

In [24]:
some_data_manager.save(copy.deepcopy(new_datapoint), overwrite_id=True, unique_id=False)

In [25]:
# print all data
data_dict = some_data_manager.load("miwow")
print("-"*70)
for key in data_dict.keys(): # type: ignore
    if key in ["RRI", "MAD", "SLP"]:
        print(key, data_dict[key][:10], data_dict[key].shape) # type: ignore
    elif key in ["RRI_windows", "MAD_windows"]:
        print(key, data_dict[key][0][:10], data_dict[key].shape) # type: ignore
    elif key in ["SLP_windows"]:
        print(key, data_dict[key][0], data_dict[key].shape) # type: ignore
    else:
        print(key, data_dict[key]) # type: ignore
print("-"*70)

----------------------------------------------------------------------
ID miwow
RRI [5. 1. 3. 5. 3. 1. 5. 5. 4. 3.] (144000,)
MAD [1 1 2 5 3 2 2 3 3 3] (36000,)
SLP [ 1  0 -1  1  0  3  1 -1  3  2] (1200,)
RRI_windows [5. 1. 3. 5. 3. 1. 5. 5. 4. 3.] (1197, 480)
MAD_windows [1. 1. 2. 5. 3. 2. 2. 3. 3. 3.] (1197, 120)
SLP_windows 1.0 (1197,)
----------------------------------------------------------------------


### Removing signals transformed to windows

As you might want to remove all transformations to windows for various reasons, you might notice that this is
not possible using the remove() function, instead use:

In [26]:
some_data_manager.remove_signals_in_windows()
file_info_after_removing_windows = some_data_manager.file_info

In [27]:
# print all data
for dict in some_data_manager:
    print("-"*70)
    for key in dict.keys():
        if key in ["RRI", "MAD", "SLP"]:
            print(key, dict[key][:10], dict[key].shape)
        elif key in ["RRI_windows", "MAD_windows"]:
            print(key, dict[key][0][:10], dict[key].shape)
        elif key in ["SLP_windows"]:
            print(key, dict[key][0], dict[key].shape)
        else:
            print(key, dict[key])
print("-"*70)

----------------------------------------------------------------------
ID 4_shift_3780s_x2
MAD [5 5 2 1 3 3 3 4 5 4] (36000,)
SLP [-1  3 -1  3  3  0  2  2  3  2] (1200,)
RRI [1.  3.  1.  3.  4.  3.  5.  1.5 1.  4. ] (144000,)
----------------------------------------------------------------------
ID 4
RRI [4.  2.5 4.  2.5 2.  2.5 5.  1.5 5.  2.5] (144000,)
MAD [3 5 2 3 3 3 2 5 2 1] (36000,)
SLP [ 2 -1  3  2  2  3  0  2  1 -1] (1200,)
----------------------------------------------------------------------
ID 4_shift_3780s_x1
RRI [5.  2.  3.  3.5 2.  1.5 2.  4.5 2.  2.5] (144000,)
MAD [4 4 3 4 1 3 2 1 1 1] (36000,)
SLP [2 3 2 0 1 3 0 0 2 0] (1200,)
----------------------------------------------------------------------
ID miwow
RRI [5. 1. 3. 5. 3. 1. 5. 5. 4. 3.] (144000,)
MAD [1 1 2 5 3 2 2 3 3 3] (36000,)
SLP [ 1  0 -1  1  0  3  1 -1  3  2] (1200,)
----------------------------------------------------------------------


See the change in the file information:

In [28]:
for key in file_info_after_transforming.keys():
    print(f"{key} (before removing / after removing): {file_info_after_transforming[key]} / {file_info_after_removing_windows[key]}")

RRI_frequency (before removing / after removing): 4 / 4
MAD_frequency (before removing / after removing): 1 / 1
SLP_frequency (before removing / after removing): 0.03333333333333333 / 0.03333333333333333
sleep_stage_label (before removing / after removing): {'wake': '0', 'LS': '1', 'DS': '2', 'REM': '3', 'artifect': '-1'} / {'wake': '0', 'LS': '1', 'DS': '2', 'REM': '3', 'artifect': '-1'}
signal_length_seconds (before removing / after removing): 36000 / 36000
wanted_shift_length_seconds (before removing / after removing): 3600 / 3600
absolute_shift_deviation_seconds (before removing / after removing): 1800 / 1800
signal_in_windows (before removing / after removing): True / False
number_windows (before removing / after removing): 1197 / 1197
window_duration_seconds (before removing / after removing): 120 / 120
overlap_seconds (before removing / after removing): 90 / 90
pad_target_with (before removing / after removing): 0 / 0
pad_feature_with (before removing / after removing): 0 / 0
pr

As you see: 'signal_to_windows' changed to 'False', meaning the signal to windows transformation won't be
applied when saving data. Additionally, the keys: 'number_windows', 'window_duration_seconds', 
'overlap_seconds' and 'priority_order' will be changed back to the default values.

### Train-, Validation-, Test- Split

Of course, we aim to train a machine learning model with the data handled by this class. So, we want to
be able to separate the data into training-, validation- and test- pids.

First, let's create a new file and add some more data:

In [29]:
many_files_data_manager = SleepDataManager(file_path = "Sleep_Data/Data.pkl")

add_number_datapoints = 100

# optimal signal (fitting sampling frequencies and length):
signal_time_in_seconds = 10 * 3600
rri_frequency = 4
mad_frequency = 1
slp_frequency = 1/30

random_sleep_stage_labels = {"wake": [0, 1], "LS": [2], "DS": [3], "REM": [5], "artifect": ["other"]}

for i in range(add_number_datapoints):
    rri_signal = np.array([random.randint(1, 5) for i in range(int(signal_time_in_seconds * rri_frequency))], dtype=np.float64)
    mad_signal = [random.randint(1, 5) for i in range(int(signal_time_in_seconds * mad_frequency))]
    slp_signal = [random.randint(1, 5) for i in range(int(signal_time_in_seconds * slp_frequency))]

    decide_what_data_to_add = random.randint(0, 2)

    if decide_what_data_to_add == 0:
        new_datapoint = {
            "ID": str(i),
            "RRI": rri_signal,
            "RRI_frequency": rri_frequency,
            "MAD": mad_signal,
            "MAD_frequency": mad_frequency,
            "SLP": slp_signal,
            "SLP_frequency": slp_frequency,
            "sleep_stage_label": random_sleep_stage_labels
        } # optimal data (rri and mad to slp)
    elif decide_what_data_to_add == 1:
        new_datapoint = {
            "ID": str(i),
            "RRI": rri_signal,
            "RRI_frequency": rri_frequency,
            "MAD": mad_signal,
            "MAD_frequency": mad_frequency,
        } # invalid data (no target: slp)
    else:
        new_datapoint = {
            "ID": str(i),
            "RRI": rri_signal,
            "RRI_frequency": rri_frequency,
            "SLP": slp_signal,
            "SLP_frequency": slp_frequency,
            "sleep_stage_label": random_sleep_stage_labels
        } # only rri to slp
    
    many_files_data_manager.save(new_datapoint, overwrite_id=False)

print(f"Number of datapoints in file: {len(many_files_data_manager)}")

Number of datapoints in file: 100


Now create separate files where training-, validation- and test- data is stored:

Data that can not be used to train the network (i.e. missing "RRI" and "SLP") will be left in the main file. 
        
As we can manage data with "RRI" and "MAD" and data with "RRI" only, the algorithm makes sure
that only one of the two types of data is used (the one with more samples). The other type will 
be left in the main file.

In [30]:
many_files_data_manager.separate_train_test_validation(
    train_size = 0.8, 
    validation_size = 0.1, 
    test_size = 0.1, 
    random_state = None, 
    shuffle = True
)


Attention: 28 datapoints with MAD signal will be left in the main file.


The individual files can be accessed by another instance of this class. 

ATTENTION:  

-   The instances on all files will have reduced functionality from now on. As the data should
    be fully prepared for the network now, the instances are designed to only load data and
    not save or edit it.

-   The functionality of the instance on the main file is not as restricted as the ones on the
    training, validation, and test files. The main file instance can additionally save data
    (only to main file, won't be forwarded to training, validation, or test files), reshuffle 
    the data in the secondary files or pull them back into the main file for further processing.

Let's access our training-, validation- and test- data:

In [31]:
main_file_info = many_files_data_manager.file_info

train_data_manager = SleepDataManager(file_path = main_file_info["train_file_path"])
validation_data_manager = SleepDataManager(file_path = main_file_info["validation_file_path"])
test_data_manager = SleepDataManager(file_path = main_file_info["test_file_path"])

Let's compare the individual file info:

In [32]:
train_file_info = train_data_manager.file_info
validation_file_info = validation_data_manager.file_info
test_file_info = test_data_manager.file_info

equal = True
for key in main_file_info.keys():
    if main_file_info[key] != train_file_info[key] or main_file_info[key] != validation_file_info[key] or main_file_info[key] != test_file_info[key]:
        equal = False
        break

if equal:
    print("Main file info is the same as train, validation and test file info!")

print("\nUniform file information:")
print("-"*60)
for key in main_file_info.keys():
    print(f"{key}: {main_file_info[key]}")

Main file info is the same as train, validation and test file info!

Uniform file information:
------------------------------------------------------------
RRI_frequency: 4
MAD_frequency: 1
SLP_frequency: 0.03333333333333333
sleep_stage_label: {'wake': '0', 'LS': '1', 'DS': '2', 'REM': '3', 'artifect': '-1'}
signal_length_seconds: 36000
wanted_shift_length_seconds: 3600
absolute_shift_deviation_seconds: 1800
signal_in_windows: False
number_windows: 1197
window_duration_seconds: 120
overlap_seconds: 90
pad_target_with: 0
pad_feature_with: 0
priority_order: [0, 1, 2, 3, 5, -1]
train_val_test_split_applied: True
main_file_path: Sleep_Data/Data.pkl
train_file_path: Sleep_Data/Data_training_pid.pkl
validation_file_path: Sleep_Data/Data_validation_pid.pkl
test_file_path: Sleep_Data/Data_test_pid.pkl


Access data:

In [33]:
print("Length of each dataset:")
print("-"*30)
print(f"Main: {len(many_files_data_manager)}")
print(f"Train: {len(train_data_manager)}")
print(f"Validation: {len(validation_data_manager)}")
print(f"Test: {len(test_data_manager)}")

Length of each dataset:
------------------------------
Main: 60
Train: 32
Validation: 4
Test: 4


In [34]:
for datapoint in validation_data_manager:
    print(datapoint.keys())

dict_keys(['ID', 'RRI', 'SLP'])
dict_keys(['ID', 'RRI', 'SLP'])
dict_keys(['ID', 'RRI', 'SLP'])
dict_keys(['ID', 'RRI', 'SLP'])


In [35]:
for datapoint in test_data_manager:
    print(datapoint.keys())

dict_keys(['ID', 'RRI', 'SLP'])
dict_keys(['ID', 'RRI', 'SLP'])
dict_keys(['ID', 'RRI', 'SLP'])
dict_keys(['ID', 'RRI', 'SLP'])


In [36]:
some_datapoint = train_data_manager.load(random.randint(0, len(train_data_manager)))

for key in some_datapoint.keys():
    print(f"{key}: {some_datapoint[key]}")

ID: 20
RRI: [5. 4. 3. ... 2. 4. 3.]
SLP: [ 0 -1  0 ...  1  1  0]


Of course, you can always reshuffle the data again from the main file manager (let's assign different 
arguments to see that something happened):

In [37]:
many_files_data_manager.separate_train_test_validation(
    train_size = 0.5, 
    validation_size = 0.4, 
    test_size = 0.1, 
    random_state = None, 
    shuffle = True
)


Attention: 28 datapoints with MAD signal will be left in the main file.


In [38]:
print("Length of each dataset:")
print("-"*30)
print(f"Main: {len(many_files_data_manager)}")
print(f"Train: {len(train_data_manager)}")
print(f"Validation: {len(validation_data_manager)}")
print(f"Test: {len(test_data_manager)}")

Length of each dataset:
------------------------------
Main: 60
Train: 20
Validation: 16
Test: 4


We can also fuse the data again (do not forget to close your active data managers!):

In [39]:
many_files_data_manager.fuse_train_test_validation()

In [40]:
del train_data_manager
del validation_data_manager
del test_data_manager

In [41]:
print(len(many_files_data_manager))

100


## Introduction to the implemented functions

In [42]:
import numpy as np # type: ignore
import random
import h5py # type: ignore

In this section you can check whether the implemented functions in this project work correctly.

### Scaling number of datapoints from signal- to target- frequency:

I would highly suggest to provide data where the signals don't need to be scaled to the frequencies of the data
used to train the neural network.

If there is no other option, then so be it. Here is a demonstration of the functions that will be applied to 
your data:

#### Classification Signal

In [43]:
classification_array = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
classification_frequency = 1/20
target_frequency = 1/30

print("-"*71)
print(f"Classification Frequency: {classification_frequency} -> Target Frequency: {target_frequency}")
print("-"*71)
print("\nClassification array: ", classification_array)
print("Classification array shape: ", classification_array.shape)

reshaped_array = scale_classification_signal(
        signal = classification_array, # type: ignore
        signal_frequency = classification_frequency,
        target_frequency = target_frequency
        )

print("\nScaled array: ", reshaped_array)
print("Scaled array shape: ", reshaped_array.shape)

classification_array = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
classification_frequency = 1/50
target_frequency = 1/30

print("\n")
print("-"*71)
print(f"Classification Frequency: {classification_frequency} -> Target Frequency: {target_frequency}")
print("-"*71)
print("\nClassification array: ", classification_array)
print("Classification array shape: ", classification_array.shape)

reshaped_array = scale_classification_signal(
        signal = classification_array, # type: ignore
        signal_frequency = classification_frequency,
        target_frequency = target_frequency
        )

print("\nScaled array: ", reshaped_array)
print("Scaled array shape: ", reshaped_array.shape)

del reshaped_array, classification_array, classification_frequency, target_frequency

-----------------------------------------------------------------------
Classification Frequency: 0.05 -> Target Frequency: 0.03333333333333333
-----------------------------------------------------------------------

Classification array:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
Classification array shape:  (15,)

Scaled array:  [ 0  1  3  4  6  7  9 10 12 13]
Scaled array shape:  (10,)


-----------------------------------------------------------------------
Classification Frequency: 0.02 -> Target Frequency: 0.03333333333333333
-----------------------------------------------------------------------

Classification array:  [0 1 2 3 4 5 6 7 8]
Classification array shape:  (9,)

Scaled array:  [0 1 1 2 2 3 4 4 5 5 6 7 7 8 8]
Scaled array shape:  (15,)


#### Continuous Signal

In [44]:
continuous_array_int = np.array([0, 1, 2, 3, 4, 5])
continuous_array_float = np.array([0, 1, 2, 3, 4, 5], dtype = float)
continuous_frequency = 3
target_frequency = 4

print("-"*75)
print(f"Continuous Frequency: {continuous_frequency} -> Target Frequency: {target_frequency}")
print("-"*75)
print(f"Continuous array: {continuous_array_int} / {continuous_array_float}")
print("Continuous array shape: ", continuous_array_int.shape)

reshaped_array_int = interpolate_signal(
        signal = continuous_array_int, # type: ignore
        signal_frequency = continuous_frequency,
        target_frequency = target_frequency
        )

reshaped_array_float = interpolate_signal(
        signal = continuous_array_float, # type: ignore
        signal_frequency = continuous_frequency,
        target_frequency = target_frequency
        )

print(f"\nScaled array: {reshaped_array_int} / {reshaped_array_float}")
print("Scaled array shape: ", reshaped_array_int.shape)

print("\n")

continuous_array_int = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
continuous_array_float = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype = float)
continuous_frequency = 5
target_frequency = 4

print("-"*75)
print(f"Continuous Frequency: {continuous_frequency} -> Target Frequency: {target_frequency}")
print("-"*75)
print(f"Continuous array: {continuous_array_int} / {continuous_array_float}")
print("Continuous array shape: ", continuous_array_int.shape)

reshaped_array_int = interpolate_signal(
        signal = continuous_array_int, # type: ignore
        signal_frequency = continuous_frequency,
        target_frequency = target_frequency
        )

reshaped_array_float = interpolate_signal(
        signal = continuous_array_float, # type: ignore
        signal_frequency = continuous_frequency,
        target_frequency = target_frequency
        )

print(f"\nScaled array: {reshaped_array_int} / {reshaped_array_float}")
print("Scaled array shape: ", reshaped_array_int.shape)

del reshaped_array_int, reshaped_array_float, continuous_array_int, continuous_array_float, continuous_frequency, target_frequency

---------------------------------------------------------------------------
Continuous Frequency: 3 -> Target Frequency: 4
---------------------------------------------------------------------------
Continuous array: [0 1 2 3 4 5] / [0. 1. 2. 3. 4. 5.]
Continuous array shape:  (6,)

Scaled array: [0 1 2 2 3 4 4 5] / [0.   0.75 1.5  2.25 3.   3.75 4.5  5.  ]
Scaled array shape:  (8,)


---------------------------------------------------------------------------
Continuous Frequency: 5 -> Target Frequency: 4
---------------------------------------------------------------------------
Continuous array: [0 1 2 3 4 5 6 7 8 9] / [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
Continuous array shape:  (10,)

Scaled array: [0 1 2 4 5 6 8 9] / [0.   1.25 2.5  3.75 5.   6.25 7.5  8.75]
Scaled array shape:  (8,)


### Splitting a signal which is too long for the neural network model

A signal which is too short will be padded with zeros. No big deal. On the other hand: A signal which is too 
long will be splitted into multiple signals. To create more data, the 10 hour range will be shifted along the
signal.

This shift should not be too small, to create redundant data but also not too big, because the more data the 
better. So we try to find a shift size close to 1 hour, which lets us shift an integer amount of times
easily. 

#### Finding optimal shift size

In [45]:
signal_length_addition_hours = 2.5
desired_length_hours = 10

optimal_shift_length = calculate_optimal_shift_length(
        signal_length = (desired_length_hours + signal_length_addition_hours) * 3600, # type: ignore
        desired_length = desired_length_hours*3600, 
        wanted_shift_length = 3600,
        absolute_shift_deviation = 1800,
)

print(f"Optimal shift length for signal which is {signal_length_addition_hours} hours longer than desired length of {desired_length_hours} hours: {round(optimal_shift_length/3600, 3)} hours")

Optimal shift length for signal which is 2.5 hours longer than desired length of 10 hours: 0.833 hours


#### Splitting Signal

The above function to find optimal shift length is embedded in the following split funtion. The optimal
shift size will be estimated for every signal individually.

If there is no integer shift size in range, that lets you shift the signal so, that you perfectly enclose the
last datapoints of the long signal, then the last shift will be altered so that it does.

In [46]:
# Create random signal
frequency = 4
length_signal_seconds = 12.1 * 3600
signal = np.random.rand(int(length_signal_seconds * frequency))

# Only important parameters here:
nn_signal_seconds = 10 * 3600
shift_length_seconds = 3600
absolute_shift_deviation_seconds = 1800

signals_from_splitting, shift_length = split_long_signal(
        signal = signal, # type: ignore
        sampling_frequency = frequency,
        target_frequency = frequency,
        nn_signal_duration_seconds = nn_signal_seconds,
        wanted_shift_length_seconds = shift_length_seconds,
        absolute_shift_deviation_seconds = absolute_shift_deviation_seconds
        )

print("Shift length:", shift_length)
print(f"Shift length: {shift_length / frequency} seconds")
print("Signal shape: ", signal.shape)
print(f"Datapoints in NN: {nn_signal_seconds * frequency}")
print("Signals from splitting shape: ", signals_from_splitting.shape)

del signals_from_splitting, signal, shift_length, frequency, nn_signal_seconds, shift_length_seconds, absolute_shift_deviation_seconds

Shift length: 15120
Shift length: 3780.0 seconds
Signal shape:  (174240,)
Datapoints in NN: 144000
Signals from splitting shape:  (3, 144000)


#### Splitting signals within dictionary

In [47]:
# Create random signal
length_signal_seconds = 12.1 * 3600
rri_frequency = 4
mad_frequency = 1
rri_signal = np.random.rand(int(length_signal_seconds * rri_frequency))
mad_signal = np.random.rand(int(length_signal_seconds * mad_frequency))

data_dict = {
    "ID": "1",
    "RRI": rri_signal,
    "RRI_frequency": rri_frequency,
    "MAD": mad_signal,
    "MAD_frequency": mad_frequency,
}

new_dictionaries = split_signals_within_dictionary(
    data_dict = data_dict,
    id_key = "ID",
    valid_signal_keys = ["RRI", "MAD"],
    signal_frequencies = [rri_frequency, mad_frequency],
    signal_target_frequencies = [rri_frequency, mad_frequency],
    nn_signal_duration_seconds = 10 * 3600,
    wanted_shift_length_seconds = 3600,
    absolute_shift_deviation_seconds = 1800
)

print("Original dictionary:")
print("-"*20)
for key, value in data_dict.items():
    if key == "RRI" or key == "MAD" or key == "SLP":
        print(f"{key}: {value.shape}")
    else:
        print(f"{key}: {value}")
print("\nNew dictionaries:")
print("-"*20)
for new_dict in new_dictionaries:
    for key, value in new_dict.items():
        if key == "RRI" or key == "MAD" or key == "SLP":
            print(f"{key}: {value.shape}")
        else:
            print(f"{key}: {value}")
    print("")

Original dictionary:
--------------------
ID: 1
RRI: (174240,)
RRI_frequency: 4
MAD: (43560,)
MAD_frequency: 1

New dictionaries:
--------------------
ID: 1
RRI: (144000,)
RRI_frequency: 4
MAD: (36000,)
MAD_frequency: 1

ID: 1_shift_3780s_x1
RRI: (144000,)
RRI_frequency: 4
MAD: (36000,)
MAD_frequency: 1

ID: 1_shift_3780s_x2
RRI: (144000,)
RRI_frequency: 4
MAD: (36000,)
MAD_frequency: 1



### Reading a .h5 - file

One of the available training datasets for our neural network model are stored in a .h5 file. So we need
to be able to read it. These are the important operations:

In [49]:
shhs_dataset = h5py.File("Raw_Data/SHHS_dataset.h5", 'r')
patients = list(shhs_dataset['slp'].keys()) # type: ignore

random_patient = patients[np.random.randint(0, len(patients))]
print(f"Random patient: {random_patient}")

print(np.unique(shhs_dataset["slp"][random_patient][:])) # type: ignore

for key in ["slp", "rri"]:
    print(f"\nkey: {key}")

    patient_data = shhs_dataset[key][random_patient][:] # type: ignore
    print(f"Data shape: {patient_data.shape}") # type: ignore

    data_freq = shhs_dataset[key].attrs["freq"] # type: ignore
    print(f"Data frequency: {data_freq}")
    print(f"Inverse data frequency: {1/data_freq}") # type: ignore

    print(f"Data length: {patient_data.shape[0]/data_freq} s") # type: ignore

del shhs_dataset, patients, random_patient, key, patient_data, data_freq

Random patient: 201949_1
[0 1 2 3 5]

key: slp
Data shape: (865,)
Data frequency: 0.03333333333333333
Inverse data frequency: 30.0
Data length: 25950.0 s

key: rri
Data shape: (103800,)
Data frequency: 4
Inverse data frequency: 0.25
Data length: 25950.0 s


### Divide up a signal into overlapping windows

The hardest thing about this is, that 'window_overlap' and 'datapoints_per_window' must be chosen so that
the whole signal fits perfectly into n windows. 

Additionally, those values must be integers. This means that 'window_duration_seconds' and 'overlap_seconds'
multiplied with 'target_fequency' as well as 'sampling_frequency' must be integers. (The features and the target labels
must fit equally well into the windows, so that we can find the correlation between a feature- and target- window.)

We have the RRI and MAD values as features and the sleep phase as target classification. As we will see,
RRI and MAD values were recorded with an integer sampling frequency. While the sampling frequency of the 
sleep classification is 1/30. 

Finding window parameters that fullfill the conditions mentioned is easier than it sounds. We will always pass data
to the neural network that is 10 hours long. Now, we just need to think in seconds and find integer values
for 'window_duration_seconds' and 'overlap_seconds' that are a multiple of 30:

#### Finding optimal window_parameters:

In [50]:
find_suitable_window_parameters(
        signal_length = 10 * 3600,
        number_windows_range = (1000, 1400),
        window_size_range = (120, 180),
        minimum_window_size_overlap_difference = 30
    )

Suitable window parameters for signal of length: 36000:
-------------------------------------------------------
Number of windows: 1025, Window size: 160, Overlap: 125.0
Number of windows: 1026, Window size: 125, Overlap: 90.0
Number of windows: 1055, Window size: 164, Overlap: 130.0
Number of windows: 1056, Window size: 130, Overlap: 96.0
Number of windows: 1087, Window size: 162, Overlap: 129.0
Number of windows: 1088, Window size: 129, Overlap: 96.0
Number of windows: 1121, Window size: 160, Overlap: 128.0
Number of windows: 1122, Window size: 128, Overlap: 96.0
Number of windows: 1157, Window size: 164, Overlap: 133.0
Number of windows: 1158, Window size: 133, Overlap: 102.0
Number of windows: 1196, Window size: 150, Overlap: 120.0
Number of windows: 1197, Window size: 120, Overlap: 90.0


Our options are:

Number of windows: 1196, Window size: 150, Overlap: 120.0 \
Number of windows: 1197, Window size: 120, Overlap: 90.0

We will choose the latter, because we don't want the window_size to be too large.

#### Classification Signal

When transforming a classification signal into windows, which is supposed to be the target in the neural 
network, then each window will only be represented by the most common sleep stage. If there is a tie
between the labels, then the one with the highest priority will be chosen 

In [51]:
signal_length_seconds = 10 * 3600
frequency = 1/30
signal_length = int(signal_length_seconds * frequency)

signal = np.array([random.randint(0, 5) for _ in range(signal_length)])

signal_in_windows = signal_to_windows(
    signal = signal, # type: ignore
    datapoints_per_window = int(120 * frequency),
    window_overlap = int(90 * frequency),
    signal_type = "target",
    priority_order = [0, 1, 2, 3, 5, -1]
    )

print(f"Signal shape: {signal.shape}")
print(f"Signal in windows shape: {signal_in_windows.shape}")

del signal, signal_in_windows, signal_length_seconds, frequency, signal_length


 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labels: [4]

 Labe

#### Continuous Signal

In [52]:
signal = np.random.rand(36000)

signal_in_windows = signal_to_windows(
    signal = signal, # type: ignore
    datapoints_per_window = 120,
    window_overlap = 90,
    signal_type = "feature"
    )

print(f"Signal shape: {signal.shape}")
print(f"Signal in windows shape: {signal_in_windows.shape}")

del signal, signal_in_windows

Signal shape: (36000,)
Signal in windows shape: (1197, 120)


#### Reshape Signal

The following function will actually be applied the transform a signal into windows. It will make sure
that the data is passed correctly to the function mentioned above. 

This means it will:
- check if 'number_nn_datapoints', 'datapoints_per_window' and 'window_overlap' are integers
- check if 'datapoints_per_window' and 'window_overlap' perfectly fit into 'number_nn_datapoints'
- compare length of provided signal to length of signal in nn ('number_nn_datapoints')
    - if smaller: Pad with Zeros
    - if bigger: Print warning, but continue by cropping last datapoints
- check if signal transformed to windows has the right shape

In [57]:
random_array = np.random.rand(36000)
reshaped_array = reshape_signal_to_overlapping_windows(
    signal = random_array, # type: ignore
    sampling_frequency = 4,
    target_frequency = 4, 
    number_windows = 1197, 
    window_duration_seconds = 120, 
    overlap_seconds = 90,
    signal_type = "feature",
    nn_signal_duration_seconds = 10*3600,
    )

print(f"Random array shape: {random_array.shape}")
print(f"Reshaped array shape: {reshaped_array.shape}")

random_array = np.array([random.randint(0, 3) for _ in range(int(36000/30))])
reshaped_array = reshape_signal_to_overlapping_windows(
    signal = random_array, # type: ignore
    sampling_frequency = 1/30,
    target_frequency = 1/30, 
    number_windows = 1197, 
    window_duration_seconds = 120, 
    overlap_seconds = 90,
    signal_type = "target",
    nn_signal_duration_seconds = 10*3600,
    )

print(f"Random array shape: {random_array.shape}")
print(f"Reshaped array shape: {reshaped_array.shape}")

del random_array, reshaped_array

Random array shape: (36000,)
Reshaped array shape: (1197, 480)
Random array shape: (1200,)
Reshaped array shape: (1197,)


#### Alter Sleep Labels

Following function makes sure to keep labels unfiform.

In [58]:
slp = np.array([-2, -1, 0, 1, 2, 3, 4, 5, 6, 7])
print(slp)

current_labels = {"wake": [0, 1], "LS": [2], "DS": [3], "REM": [5], "artifect": ["other"]}
desired_labels = {"wake": 0, "LS": 1, "DS": 2, "REM": 3, "artifect": -1}

print(alter_slp_labels(
        slp_labels = slp,
        current_labels = current_labels,
        desired_labels = desired_labels,
))

[-2 -1  0  1  2  3  4  5  6  7]
[-1 -1  0  0  1  2 -1  3 -1 -1]


In [59]:
slp = np.array(["light_sleep", "deep_sleep", "deep_sleep_2", "WAKE", "REM", "bla", "blub"])
print(slp)

current_labels = {"wake": ["WAKE"], "LS": ["light_sleep"], "DS": ["deep_sleep", "deep_sleep_2"], "REM": ["REM"], "artifect": ["other"]}
desired_labels = {"wake": 0, "LS": 1, "DS": 2, "REM": 3, "artifect": -1}

print(alter_slp_labels(
        slp_labels = slp,
        current_labels = current_labels,
        desired_labels = desired_labels,
))

['light_sleep' 'deep_sleep' 'deep_sleep_2' 'WAKE' 'REM' 'bla' 'blub']
['1' '2' '2' '0' '3' '-1' '-1']


In [60]:
slp = np.array([-2, -1, 0, 1, 2, 3, 4, 5, 6, 7])
print(slp)

slp[slp>=1] = slp[slp>=1] - 1
slp[slp==4] = 3
slp[slp==5] = 0
slp[slp==-1] = 0 # set artifact as wake stage

print(slp)

[-2 -1  0  1  2  3  4  5  6  7]
[-2  0  0  0  1  2  3  3  0  6]
